In [18]:
import pandas as pd
import numpy as np
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from datetime import timedelta

In [177]:
censoreddf = pd.read_excel("Censored meters details_1605.xlsx")

In [178]:
censoreddf['Starts from'] = pd.to_datetime(censoreddf['Starts from'], format='%Y-%m-%d')

In [179]:
censoreddf.set_index('Starts from', inplace=True)

In [132]:
datemid = censoreddf[(censoreddf['MeterID'] == 'KOL-22813398')]

In [133]:
datemid.index = pd.to_datetime(datemid.index)

In [149]:
pd.to_datetime(dt1, format='%Y-%m-%d')[0]

Timestamp('2020-02-01 00:00:00')

In [169]:
KS_stats_df = pd.DataFrame(columns = ['MeterID', 'CP-date','KS-stats','Critical-KS','KS_stats > Critical_KS', 'P-value','alpha','P-value<=aplha','Outcome'])

In [180]:
for mid in censoreddf['MeterID']:
    tdf=pd.read_csv("Data_processed/"+mid+".csv")
    tdf['dates'] = tdf['dates'].str[:10]
    tdf['dates'] = pd.to_datetime(tdf['dates'], format='%Y-%m-%d')
    tdf.set_index('dates', inplace=True)
    num = tdf._get_numeric_data()
    num[num < 0] = 0
    datemid = censoreddf[censoreddf['MeterID'] == mid]
    dt1 = datemid.index
    dt2 = dt1 - timedelta(days = 1)
    dt1 = pd.to_datetime(dt1, format='%Y-%m-%d')[0]
    dt2 = pd.to_datetime(dt2, format='%Y-%m-%d')[0]
    d1 = tdf.loc[tdf.index==dt2].iloc[:, 20:74].to_numpy()
    flag = not np.any(d1)
    if flag:
        dt2 = dt1 - timedelta(days = 2)
        d1 = tdf.loc[tdf.index==dt2].iloc[:, 20:74].to_numpy()
    d2 = tdf.loc[tdf.index==dt1].iloc[:, 20:74].to_numpy()
    KS_stats = stats.ks_2samp(d1[0], d2[0])
    KS_stats1=KS_stats[0]
    KS_stats2=KS_stats[1]
    if KS_stats1 > 0.2612:
        KS_critical = 'Yes'
    else:
        KS_critical = 'No'
    if KS_stats2 < 0.05:
        KS_pval = 'Yes'
    else:
        KS_pval = 'No'
    if ((KS_critical == 'Yes') and (KS_pval == 'Yes')):
        outcome = 'Not similar'
    else:
        outcome = 'Similar'
    KS_stats_df = KS_stats_df.append({'MeterID': mid, 'CP-date': dt1,'KS-stats': KS_stats1,'Critical-KS': '0.2612','KS_stats > Critical_KS': KS_critical, 'P-value': KS_stats2,
                                'alpha': '0.05','P-value<=aplha': KS_pval,'Outcome': outcome}, ignore_index = True)
    #print(mid, dt2, d1)

In [182]:
KS_stats_df.to_excel("KS-statistics_results.xlsx", index=False)

In [119]:
def cdf(sample, x, sort = False):
    # Sorts the sample, if unsorted
    if sort:
        sample.sort()
    # Counts how many observations are below x
    cdf = sum(sample <= x)
    # Divides by the total number of observations
    cdf = cdf / len(sample)
    return cdf

In [120]:
def ks_2samp(sample1, sample2):
    # Gets all observations
    observations = np.concatenate((sample1, sample2))
    observations.sort()
    # Sorts the samples
    sample1.sort()
    sample2.sort()
    # Evaluates the KS statistic
    D_ks = [] # KS Statistic list
    for x in observations:
        cdf_sample1 = cdf(sample = sample1, x  = x)
        cdf_sample2 = cdf(sample = sample2, x  = x)
        D_ks.append(abs(cdf_sample1 - cdf_sample2))
    ks_stat = max(D_ks)
    # Calculates the P-Value based on the two-sided test
    # The P-Value comes from the KS Distribution Survival Function (SF = 1-CDF)
    m, n = float(len(sample1)), float(len(sample2))
    en = m * n / (m + n)
    p_value = stats.kstwo.sf(ks_stat, np.round(en))
    return {"ks_stat": ks_stat, "p_value" : p_value}

In [121]:
# Performs the KS normality test for two time-series
ks_2samp(d1, d2)

{'ks_stat': 0.6851851851851852, 'p_value': 3.952845765633981e-13}